In [1]:
from keras.datasets import mnist 
import matplotlib.pyplot as plt 
from keras.optimizers import Adam 
from keras import utils 
import numpy as np 
import cv2
import matplotlib.pyplot as plt 
from tensorflow.keras.models import load_model 

In [2]:
dataset = mnist
(x_train2, y_train2), (x_test2, y_test2) = dataset.load_data() #translate the data to image 

x_train2=[cv2.cvtColor(cv2.resize(i,(48,48)),cv2.COLOR_GRAY2BGR)for i in x_train2]  #x_train shape is (10000,48,48,3)
x_train2=np.concatenate([arr[np.newaxis]for arr in x_train2]).astype('float32')
x_train2=x_train2/255
x_test2=[cv2.cvtColor(cv2.resize(i,(48,48)),cv2.COLOR_GRAY2BGR)for i in x_test2]
x_test2=np.concatenate([arr[np.newaxis]for arr in x_test2]).astype('float32')
x_test2=x_test2/255

vgg = load_model('Model/vgg_mnist.h5')#load lenet5 model trained with mnist dataset 
vgg.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy']) 



In [24]:
beta = 0.322
N = 600
sample(x_test2[1],y_test2[1],vgg,N,beta)

Sampled 8601 images, 8001 outside beta, 600 inside beta
600 images inside beta have the same label, local robustness formula satisfied
maximum distance 0.322


0.322

In [ ]:
def sample(x_test,y_test,model,N, beta):
    
    count = 0
    count2 = 0
    neighbour_list=[]
    mean = np.zeros(3)  
    cov = np.eye(3)
    label = y_test
    
    while True:
        count2+=1
        p = np.random.multivariate_normal(mean, cov,(48,48)) #sample formula
        
        neighbour = p + x_test*255
        neighbour=neighbour.reshape(1,48,48,3)
        neighbour /= 255
        distance=np.linalg.norm(p) 
        distance /= 255
        #print(distance)
        if (distance < beta or distance == beta):
            
            output = model.predict(neighbour,batch_size=128)[0]
            proba = softmax(output)
            pred = np.argmax(proba)
            if(pred == label): 
                count+=1  #correct prediction
            else:  #if there are some iamges inside beta cannot have the same label, beta will be decreased
                beta = beta-0.0001
                print("Beta is reduced by 0.0001, the new beta is",beta) 
                
        if(count==N):  
            #print("Sampled %s images, %s outside beta, %s inside beta" % (count2, count2-count,count))
            print("%s images inside beta have the same label, local robustness formula satisfied" %N)
            break

    
    print("maximum distance",beta)            

    return beta

In [4]:
def softmax(x):
    e_x = np.exp(x-np.max(x))
    return e_x / e_x.sum()

In [5]:
beta = 0.3213
N = 671
sample(x_test2[1],y_test2[1],vgg,N, beta)

Sampled 14875 images, 14204 outside beta, 671 inside beta
671 images inside beta have the same label, local robustness formula satisfied
maximum distance 0.3213


0.3213

In [13]:
beta = 0.3178
N = 783
sample(x_test2[1],y_test2[1],vgg,N, beta)

783 images inside beta have the same label, local robustness formula satisfied
maximum distance 0.3178


0.3178